In [1]:
from typing import List

from oop_functions.experiment_helper import *
from oop_functions.experiment_runner import ExperimentRunner
from oop_functions.util_functions import *
from oop_functions.visualization_util import *
from oop_functions.analytics_cv_util import *

%matplotlib inline
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns

from xgboost import XGBClassifier

import itertools


In [17]:
filesuffix = 'experiment_participants_screened_first_5_XGBClassifier_for_cancer_in_next_1_years_15_trials'


In [ ]:
cv_analytics_util = load_cv_analytics_util_see_stats(filesuffix)

In [4]:
id_col = 'index'
label = 'cancer_in_next_1_years'
id_and_confidence = []
for analytics_util in cv_analytics_util.analytics_utils:
    X_test = analytics_util.data_util.test_df[[id_col, 'plco_id', label]]
    X_test_mismatch = X_test.copy()
    y_pred, y_prob = analytics_util.get_predictions() 
    X_test_mismatch[f'{label}_pred'] = y_pred
    X_test_mismatch[f'{label}_prob'] = y_prob
    X_test_mismatch = X_test_mismatch[X_test_mismatch[label] != X_test_mismatch[f'{label}_pred']]
    X_test_mismatch = X_test_mismatch[X_test_mismatch[label] == 1]
    id_and_confidence.append(X_test_mismatch)
mid_conf_predictions_2 = pd.concat(id_and_confidence)

In [30]:
# TODO: include plco_ids and other stuff
mismatches = []
for analytics_util in cv_analytics_util.analytics_utils:
    X_test_mismatch = analytics_util.get_mismatches_neightbors()
    empty_dict = {col: None for col in X_test_mismatch[0][0].columns}
    for mismatch in X_test_mismatch:
        mismatch_df = pd.concat([*mismatch[:2], pd.DataFrame(empty_dict, index=[0])])
        # mismatch_df.append(empty_dict, ignore_index=True)
        mismatches.append(mismatch_df)

mismatches = pd.concat(mismatches)

In [29]:
mismatches.to_csv('./false_negatives.csv', index=False)

In [28]:
print_df(mismatches)

+-------+-------+------------+-------------+------------+-----------+------------+------------+-----------+-----------+-----------------+----------+-----------+----------+-----------+------------+------------+----------+-------------+-----------------+-----------------+--------------+------------+----------------+-----------------+------------------------+----------+------------+------------+-------------+--------------+-----------+---------------------+-----------+-----------+-----------+------------+------------+--------------+--------------+--------+-----------+----------+------------+-----------------+-----------+-------------+--------------+-----------+--------------+-------------+------------+--------------+----------+-------------+--------------+------------+-----------+------------+-----------+-----------------------+-------------+-------------+---------+---------------------+-----------+-----------+-----------+-----------+-----------+------------+------------+-----------------

In [11]:
false_negatives = pd.concat(np.array(X_test_mismatch)[:, 0])
false_negatives = false_negatives[false_negatives['cancer_in_next_1_years_prob'] < 0.5]
print_df(false_negatives)

+------+-------+------------+-------------+-------+---------+-------+------------+-----------+-----------+-----------------+----------+-----------+----------+-----------+------------+------------+----------+-------------+-----------------+-----------------+--------------+------------+----------------+-----------------+------------------------+----------+------------+------------+-------------+---------+-----------+---------------------+-----------+---------+---------+----------+----------+--------------+--------------+--------+---------+----------+------------+-----------------+-----------+-------------+--------------+-----------+--------------+------------+------------+--------------+----------+-------------+--------------+------------+-----------+--------+----------+-----------------------+-------------+-------------+---------+---------------------+-----------+-----------+-----------+-----------+-----------+----------+----------+-----------------+-----------+---------+-----------+---

In [12]:
len(false_negatives)

5

In [13]:
print_df(pd.concat(X_test_mismatch[1][:2]))

+-------+-------+------------+-------------+-------+---------+-------+------------+-----------+-----------+-----------------+----------+-----------+----------+-----------+------------+------------+----------+-------------+-----------------+-----------------+--------------+------------+----------------+-----------------+------------------------+----------+------------+------------+-------------+---------+-----------+---------------------+-----------+---------+---------+----------+----------+--------------+--------------+--------+---------+----------+------------+-----------------+-----------+-------------+--------------+-----------+--------------+------------+------------+--------------+----------+-------------+--------------+------------+-----------+--------+----------+-----------------------+-------------+-------------+---------+---------------------+-----------+-----------+-----------+-----------+-----------+----------+----------+-----------------+-----------+---------+-----------+--

In [14]:
# drop col if the feature value of patient that was misclassified does not appear in the list of feature values of patients that were classified
df = pd.concat(X_test_mismatch[1][:2])
for col in df.columns:
    if df[col].iloc[0] in df[col].iloc[list(range(1, 6))].to_list():
        df.drop(col,inplace=True,axis=1)
print_df(df)
summary = summarize_features(df)

+-------+-------+----------+----------+------------+-----------------+---------------------+-----------+------------+-----------+----------+--------------+---------------+---------------+-------------+--------------+--------------+---------------+------------+-------------+-----------+-----------+----------+-----------+--------------+------------+--------------------------+-------------------------------+-------------------------------+------------+
|       |   age |   bmi_20 |   bmi_50 |   bmi_curr |   breast_fh_age |   colon_comorbidity |   fmenstr |   hearta_f |   llong_p |   lvol_p |   ovary_diam |   ovary_diaml |   ovary_diamr |   ovary_vol |   ovary_voll |   ovary_volr |   ovcyst_diam |   polyps_f |   rantero_p |   rlong_p |   rtran_p |   rvol_p |   sisters |   weight20_f |   weight_f |   cancer_in_next_1_years |   cancer_in_next_1_years_pred |   cancer_in_next_1_years_prob |   distance |
|-------+-------+----------+----------+------------+-----------------+---------------------+

In [15]:

df = pd.concat(X_test_mismatch[4][:2])
for col in df.columns:
    if len(df[col].unique()) <= 2:
        df.drop(col,inplace=True,axis=1)
print_df(df)
summary = summarize_features(df)
print_df(summary)

+-------+----------+----------+------------+----------+------------+-----------------+----------+-----------+------------+-------------+-----------+-----------+-----------+----------+-----------+--------------+---------------+---------------+-------------+--------------+--------------+---------------+---------------+-------------+-------------+-----------+-----------+-----------+-----------+----------+----------+-----------+--------------+--------------+------------+------------+
|       |   bmi_20 |   bmi_50 |   bmi_curr |   bq_age |   brothers |   ca125ii_level |   educat |   fmenstr |   height_f |   lantero_p |   llong_p |   lmenstr |   ltran_p |   lvol_p |   occupat |   ovary_diam |   ovary_diaml |   ovary_diamr |   ovary_vol |   ovary_voll |   ovary_volr |   ovcyst_diam |   ovcyst_volr |   rantero_p |   rantero_q |   rlong_p |   rlong_q |   rtran_p |   rtran_q |   rvol_p |   rvol_q |   sisters |   weight20_f |   weight50_f |   weight_f |   distance |
|-------+----------+----------